In [1]:
import nltk
import numpy as np 

from HMM import unsupervised_HMM
from HMM_helper import (
    text_to_wordcloud,
    states_to_wordclouds,
    parse_observations,
    sample_sentence,
    visualize_sparsities,
    animate_emission,
    sample_rhyme
)

In [2]:
raw = open("data/shakespeare.txt").read()

In [3]:
word_dict = []
syllables = {}

# Wrangle syllable data
with open('data/Syllable_dictionary.txt') as file:
    for line in file:
        word = line.strip('\n').split(' ')
        word_dict.append(word[0])
        for syl in word[1:]:
            if syl not in syllables:
                syllables[syl] = [word_dict[-1]]
            else:
                syllables[syl].append(word_dict[-1])
file.close()

word_dict = ["\n"] + word_dict

In [4]:
sonnets = []
rhymes = []

start = 0

# Wrangle sonnets using syllables
with open('data/shakespeare.txt') as file:
    for ind, line in enumerate(file):
        son = []
        if line == '\n':
            continue
        if line.strip().isdigit():
            start = ind
            continue
        words = line.strip().split(' ')
        
        # Add word to list
        for word in words:
            word = word.lower()
            
            # If it's a word in our word_dict
            if word in word_dict:
                son.append(word_dict.index(word))
            
            # Else check if we can strip off the punct
            elif word[1:] in word_dict:
                if word[0] not in word_dict:
                    word_dict.append(word[0])
                    syllables['0'].append(word[0])
                
                # Add punctuation and word
                son.append(word_dict.index(word[0]))
                son.append(word_dict.index(word[1:]))
                
            elif word[:-1] in word_dict:
                if word[-1] not in word_dict:
                    word_dict.append(word[-1])
                    syllables['0'].append(word[-1])
                
                # Add word and punctuation
                son.append(word_dict.index(word[:-1]))
                son.append(word_dict.index(word[-1]))
                
            continue
        
        # Add rhyme pairs
        if len(word_dict[son[-1]]) > 1:
            last_word = son[-1]
        else: last_word = son[-2]
        
        # If lines 1, 2, 5, 6, 9, 10
        if (ind - start) < 3 or ((ind - start) > 4 and (ind - start) < 7) or ((ind - start) > 8 and (ind - start) < 11):
            rhymes.append(last_word)
        
        # If line 13
        elif (ind - start) == 13:
            rhymes.append(last_word)
            
        # If line 14
        elif (ind - start) == 14:
            rhymes.append((last_word, rhymes[-1]))
            rhymes[-2] = (rhymes[-2], last_word)
        
        # If any other line
        else:
            rhymes.append((last_word, rhymes[-2]))
            rhymes[-3] = (rhymes[-3], last_word)
            
        # Add reversed line to sonnets
        sonnets.append(son[::-1])
        
file.close()

In [5]:
for key in syllables:
    syllables[key] = [word_dict.index(word) for word in syllables[key]]

In [6]:
hmm_rhyme = unsupervised_HMM(sonnets, 2, 1, syllables, rhymes)

In [7]:
token_dict = {word_dict[ind]:ind for ind in range(len(word_dict))}

In [8]:
print('Sample Sentence:\n====================')
print(sample_rhyme(hmm_rhyme, token_dict))

Sample Sentence:
Art lose date boat days and self-same but be , 
 whose graces no is at tongue born sleep know . 
 of by nights is again idolatry : 
 to most live though of i fearful swift show , 
 the catch that eyes as sure bristly devised , 
 doth conspire against in beauty's child live , 
 fawn to thou gone neglected sympathized , 
 another count gilded i let achieve , 
 grows i conquest pace proves time's when where catch , 
 but i thine it that fond ear his o'er-read , 
 affords me thing love so or bring , dispatch , 
 with precious interchange that dull man dead : 
 will with desire should conceit thy blood there , 
 will smiling would of very that my where , 
...


# Rhyming

In [9]:
word_dict[1735]

'morrow'